In [1]:
import pandas as pd
import numpy as np
import re

In [ ]:
import torch
torch.cuda.device_count()

1

In [2]:
df = pd.read_json('../speeches.json',orient='records')

In [3]:
df.iloc[0].to_dict()['segments']

{'segment': 0,
 'tag': 'RESOCONTO',
 'start': 'RESOCONTO STENOGRAFICO',
 'president': 0.128916308276516,
 'page': 0.032191545690914,
 'len': 2,
 'score': 1.484956310433453,
 'year': 2015,
 'month': 6,
 'day': 8,
 'leg': 17,
 'id': 370,
 'convocation': 25231,
 'date': '2015-06-08 00:00:00',
 'title': 'Seduta di lunedì 8 giugno 2015',
 'text': 'RESOCONTO STENOGRAFICO 437. ',
 'presidenza': False,
 'speech': False}

{"segment": 0, "tag": "RESOCONTO", "start": "RESOCONTO STENOGRAFICO", "president": 0.12891630827651634, "page": 0.032191545690914675, "len": 2, "score": 1.4849563104334538, "year": 2015, "month": 6, "day": 8, "leg": 17, "id": 370, "convocation": 25231, "date": "2015-06-08 00:00:00", "title": "Seduta di luned\u00ec 8 giugno 2015", "text": "RESOCONTO STENOGRAFICO 437. ", "presidenza": false, "speech": false},

In [4]:
from IPython.display import clear_output

data = []
l = len(df.index)
for index in range(l):
    info = df.iloc[index].to_dict()['segments']
    data.append(info)
    if index%1000==0:
        clear_output(wait=True)
        print('done '+str(index)+'/'+str(l)+'    '+str((index/l)*100)+'%')

clear_output(wait=False)
text = pd.DataFrame.from_dict(data)
text = text[text['score']>2.5]

In [5]:
text.iloc[121235]

segment                                                        352
tag                                                      FRANCESCO
start                               FRANCESCO MONACO. La prima do-
president                                                 0.055345
page                                                       0.06436
len                                                             20
score                                                     4.471383
year                                                          2000
month                                                            9
day                                                             26
leg                                                             13
id                                                              33
convocation                                                  22923
date                                           2000-09-26 00:00:00
title                   Seduta n. 777 di martedì 26 settembre 

In [6]:
text = text[['text','persona','date','surname','name']]

## Info about deputy (Legislator) and ministers

In [2]:
# load info about deputy
df1 = pd.read_csv('parlamentari0x.csv')
df2 = pd.read_csv('parlamentari1x.csv')
dfp = pd.concat([df1, df2], ignore_index=True)
# transform date from int to date format
dfp['inizioMandato'] = pd.to_datetime(dfp['inizioMandato'], format="%Y%m%d")
dfp['fineMandato'] = dfp['fineMandato'].fillna(20990101)  # if person is currently active use placeholder as end date 
dfp['fineMandato'] = pd.to_datetime(dfp['fineMandato'], format="%Y%m%d")
# loan info about senators
df1 = pd.read_csv('senato0x.csv')
df2 = pd.read_csv('senato1x.csv')
dfs = pd.concat([df1, df2], ignore_index=True)
dfs['inizioMandato'] = pd.to_datetime(dfs['inizioMandato'], format="%Y%m%d")
dfs['fineMandato'] = dfs['fineMandato'].fillna(20990101)  # if person is currently active use placeholder as end date 
dfs['fineMandato'] = pd.to_datetime(dfs['fineMandato'], format="%Y%m%d")
# load info about ministers
df1 = pd.read_csv('ministri0x.csv')
df2 = pd.read_csv('ministri1x.csv')
dfm = pd.concat([df1, df2], ignore_index=True)
# rename some colums (to be abble to concat)
dfm = dfm.rename(columns={"d": "persona"})
dfp = dfp.rename(columns={"inizioMandato": "dataInizio", "fineMandato": "dataFine"})
dfs = dfs.rename(columns={"inizioMandato": "dataInizio", "fineMandato": "dataFine"})
# replace ministers link to it's personal link (have same numerical number, need only to change prefix and remove '_xx')
for i, d in dfm.iterrows():
    m = re.search('deputato.rdf/d(.+?)_', d['persona'])
    dfm.at[i,'persona'] = 'http://dati.camera.it/ocd/persona.rdf/p'+m.group(1)
# transform date from int to date format   
dfm['dataInizio'] = pd.to_datetime(dfm['dataInizio'], format="%Y%m%d")
dfm['dataFine'] = dfm['dataFine'].fillna(20990101)
dfm['dataFine'] = pd.to_datetime(dfm['dataFine'], format="%Y%m%d")
# need only some colums
dfp = dfp[['persona','cognome','nome','dataInizio','dataFine']]
dfm = dfm[['persona','cognome','nome','dataInizio','dataFine','carica','nomeOrganoGoverno']]
# final concat into one df
filtered_personnel = pd.concat([dfp, dfm, dfs], ignore_index=True)
filtered_personnel = filtered_personnel.rename(columns={"cognome": "surname", "nome": "name"})
#filtered_personnel =  filtered_personnel[['persona','surname','name','dataInizio','dataFine','carica','nomeOrganoGoverno']]
filtered_personnel

,persona,surname,name,dataInizio,dataFine,carica,nomeOrganoGoverno
0,http://dati.camera.it/ocd/persona.rdf/p10000,RESCIGNO,MATTEO,1948-04-27,1953-06-24,NaN,NaN
1,http://dati.camera.it/ocd/persona.rdf/p1000,BONOMI,PAOLO,1948-04-27,1953-06-24,NaN,NaN
2,http://dati.camera.it/ocd/persona.rdf/p1000,BONOMI,PAOLO,1953-06-15,1958-06-11,NaN,NaN
3,http://dati.camera.it/ocd/persona.rdf/p1000,BONOMI,PAOLO,1958-06-03,1963-05-15,NaN,NaN
4,http://dati.camera.it/ocd/persona.rdf/p10010,RICCI,MARIO,1948-04-23,1953-06-24,NaN,NaN
...,...,...,...,...,...,...,...
21016,http://dati.camera.it/ocd/persona.rdf/p306702,TOSATO,PAOLO,2014-07-02,2099-01-01,NaN,NaN
21017,http://dati.camera.it/ocd/persona.rdf/p306900,BOCCARDI,MICHELE,2015-09-08,2099-01-01,NaN,NaN
21018,http://dati.camera.it/ocd/persona.rdf/p307896,SEGRE,LILIANA,2018-01-19,2099-01-01,NaN,NaN
21019,http://dati.camera.it/ocd/persona.rdf/p308320,ALESSANDRINI,VALERIA,2020-03-12,2099-01-01,NaN,NaN


In [8]:
text2 = text[['text','persona','date','surname','name']].sample(1000)
text2 = text2.reset_index(drop=True)

In [9]:
text = text.reset_index(drop=True)

In [10]:
filtered_personnel.loc[filtered_personnel['surname'] == "NARO"]

,persona,surname,name,dataInizio,dataFine,carica,nomeOrganoGoverno
6865,http://dati.camera.it/ocd/persona.rdf/p300260,NARO,GIUSEPPE,2001-05-21,2006-04-27,NaN,NaN
7046,http://dati.camera.it/ocd/persona.rdf/p300260,NARO,GIUSEPPE,2008-04-23,2013-03-14,NaN,NaN
19294,http://dati.camera.it/ocd/persona.rdf/p300260,NARO,GIUSEPPE,2006-04-21,2008-04-28,NaN,NaN


In [11]:
from collections import Counter
def words(text): 
    text = text.upper().replace("À","A'").replace("È","E'").replace("É","E'").replace("Ì","I'").replace("Í","I'").replace("Ò","O'").replace("Ó","O'").replace("Ù","U'").replace("Ú","U'").replace("Ü","U'")
    return re.findall(r"[A-Z'-]+", text.upper())
SURNAME = Counter(words(' '.join(filtered_personnel['surname'])))
print("SURNAME done")
NAME = Counter(words(' '.join(filtered_personnel['name'])))
print("NAME done")
t = ' '.join(text['text'])

#t = t.replace("À","A'")
#t = t.replace("È","E'")
#t = t.replace("È","E'")
#t = t.replace("É","E'")
#t = t.replace("Ì","I'")
#t = t.replace("Ò","O'")
#t = t.replace("Ó","O'")
#t = t.replace("Ù","U'")
TEXT = Counter(words(t))
print("TEXT done")
WORDS = SURNAME

SURNAME done
NAME done
TEXT done


In [12]:
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = "abcdefghijklmnopqrstuvwxyz'".upper()
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [13]:
split_df = np.array_split(text, 1000)

STANZA

In [14]:
import stanza
# Download an English model into the default directory
print("Downloading Italian model...")
stanza.download('it')

C:\Users\Anton\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2022-09-24 20:01:52 INFO: Downloading default packages for language: it (Italian) ...
2022-09-24 20:01:53 INFO: File exists: C:\Users\Anton\stanza_resources\it\default.zip
2022-09-24 20:01:56 INFO: Finished downloading models and saved to C:\Users\Anton\stanza_resources.


In [15]:
print("Building a Itlaian pipeline...")
it_nlp = stanza.Pipeline('it', processors='tokenize,mwt,ner')

2022-09-24 20:01:56 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


Building a Itlaian pipeline...


2022-09-24 20:02:18 INFO: Loading these models for language: it (Italian):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |
| ner       | fbk      |

2022-09-24 20:02:18 INFO: Use device: gpu
2022-09-24 20:02:18 INFO: Loading: tokenize
2022-09-24 20:02:25 INFO: Loading: mwt
2022-09-24 20:02:25 INFO: Loading: ner
2022-09-24 20:02:26 INFO: Done loading processors!


In [16]:
text2.iloc[1]['text']

'BARDI RICCARDO, GHISLANDI, LUZ'

In [19]:
doc = it_nlp(text2.iloc[1]['text'])
print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')

entity: BARDI RICCARDO	type: PER
entity: GHISLANDI	type: PER
entity: LUZ	type: PER


RENZO PATRIA
Zacchera


In [20]:
def find_by_surname(text,personnel, corr):
    import re
    import pandas as pd
    data = []
    for i, s in personnel.iterrows():
        if re.search(r"\b" + re.escape(s['surname']) + r"\b", text):
            data.append(s)
        #if s['surname'] in text:
        #    data.append(s)
    sur = pd.DataFrame(data)
    #if len(pd.unique(sur["surname"])) > 1 and len(words) > 1: # if more then one surename in result, reduce up to first in text
        #len(words) just as hedge against error
    #    sur = find_by_surname(words[0:-1],sur, False)
    if len(sur.index) == 0 and not corr:
        WORDS = SURNAME+TEXT
        word = regex.sub('', text).split()
        for i, w in enumerate(word):
            word[i] = correction(w)
        sur = find_by_surname(' '.join(word),personnel,True)
    return sur

def find_by_surname_2(text,personnel, corr):
    import pandas as pd
    import segment
    data = []
    seg = segment2_sur.segment(text)
    sur = pd.DataFrame({'A' : []})
    #print(seg)
    for i, s in enumerate(seg):
        p = personnel[(personnel["surname"] == s.upper())]
        sur = pd.concat([sur,p])
    #if len(pd.unique(sur["surname"])) > 1 and len(words) > 1: # if more then one surename in result, reduce up to first in text
        #len(words) just as hedge against error
    #    sur = find_by_surname(words[0:-1],sur, False)
    if len(sur.index) == 0 and not corr:
        WORDS = SURNAME+TEXT
        word = regex.sub('', text).split()
        for i, w in enumerate(word):
            word[i] = correction(w)
        sur = find_by_surname(' '.join(word),personnel,True)
    return sur

def find_by_name(text,personnel,corr):
    import pandas as pd
    data = []
    for i, s in personnel.iterrows():
        if s['name'] in text:
            data.append(s)
    sur = pd.DataFrame(data)
    #if len(pd.unique(sur["surname"])) > 1 and len(words) > 1: # if more then one surename in result, reduce up to first in text
        #len(words) just as hedge against error
    #    sur = find_by_surname(words[0:-1],sur, False)
    #if len(sur.index) == 0 and not corr:
    #    WORDS = NAME+TEXT
    #    word = regex.sub('', text).split()
    #    for i, w in enumerate(word):
    #        word[i] = correction(w)
    #    sur = find_by_surname(' '.join(word),personnel,True)
    return sur

In [21]:
def find_person(text, date, personnel):
    global phase
    import segment
    import re
    import pandas as pd
    import string
    global l
    phase = "char_replace"
    regex = re.compile("[^a-zA-Z '()-]")
    text = text.replace("À","A'")
    text = text.replace("È","E'")
    text = text.replace("É","E'")
    text = text.replace("Ì","I'")
    text = text.replace("Ò","O'")
    text = text.replace("Ó","O'")
    text = text.replace("Ù","U'")
    #sentences = re.split('[,.]', text)
    #while(not sentences==[]):
    #    sentences[0] = regex.sub('', sentences[0])
    #    if (re.match("^[a-zA-Z\s'()-]+$", sentences[0])):
    #        if sentences[0].isspace():
    #            sentences.pop(0)
    #        else:
    #            break
    #    else:
    #        sentences.pop(0)
    #if not sentences==[]:
    #    text = sentences[0].lstrip()

    #print(text)
    # take only up to 20 first words
    word = text.split()
    su = 0
    le = 0
    for w in word:
        if (re.match("^[a-zA-Z\s'()-]+$", w)):
            su += len(w)
            le += 1
    if le > 0:
        average = su/le
    else:
        average = 0
    if average < 4:
        if len(word) >= 120: #avarage 6-8 letters in italian word (6*20 = 120), but segment have limit of 100
            seg = segment.segment(''.join(word[0:60]))
        else:
            seg = segment.segment(''.join(word))
        text = ' '.join(seg)
    elif len(word) >= 20:
        text = ' '.join(word[0:20])
    else:
        text = ' '.join(word)
    print('Text:')
    print(text)
    print('====================')
    chunk_list = []
    doc = it_nlp(text.upper())
    for ent in doc.sentences[0].ents:
        chunk_list.append(ent.text)
    chunked_name = ' '.join(chunk_list)
    print(chunked_name)
    print('====================')
    phase = "date_search"
    date = personnel[(personnel["dataInizio"] <= date) & (personnel["dataFine"] >= date)]
    if len(date.index) >= 1:
        phase = "sur_search"
        sur = find_by_surname(chunked_name.upper(), date, False)
        print(sur)
        if len(sur.index) == 0:
            return None
        elif (len(sur.index) == 1) or (len(pd.unique(sur['persona'])) == 1):
            return sur.iloc[0]['persona']
        else:
            phase = "name_search"
            name = find_by_name(text.upper(), sur, False)
            #print('-------------------')
            #print(name)
            if len(name.index) == 1 or (len(name.index) > 1 and len(pd.unique(name['persona']))==1):
                return name.iloc[0]['persona']
            if len(sur['surname'].value_counts()) > 1:
                word = regex.sub('', text.upper()).split()
                sur2 = pd.DataFrame({'A' : []})
                i = 0
                t = ''
                while(sur2.empty and (i < 20 and i < len(word))): #limit of frase up to 20 words
                    t = t + ' ' + word[i]
                    sur2 = find_by_surname(t,sur,False) # check what surname is appering first in text 
                    i+=1
                sur = sur2
                if sur.empty:
                    return None
            #print(sur)
            if len(pd.unique(sur['persona'])) == 1:
                return sur.iloc[0]['persona']
            else:
                phase = "name_search"
                name = find_by_name(text.upper(), sur, False)
                #print('-------------------')
                #print(name)
                if len(name.index) == 1 or (len(name.index) > 1 and len(pd.unique(name['persona']))==1):
                    return name.iloc[0]['persona']
    else:#No person found with that surename with correction
        return None

In [23]:
%%time

l = len(text2.index)
res = []

regex = re.compile("[^a-zA-Z -']")

for index, row in text2.iterrows():
    #DEBUG
    #if index%100==0:
    clear_output(wait=True)
    print('done '+str(index)+'/'+str(l)+'    '+str((index/l)*100)+'%')
    #DEBUG
    global phase
    phase = "search"
    person = find_person(row['text'], row['date'], filtered_personnel)
    phase = "take person data"
    predicted = filtered_personnel.loc[filtered_personnel['persona'] == person]
    if person is not None:
        pred_surname = predicted.iloc[0]['surname']
        pred_name = predicted.iloc[0]['name']
    else:
        pred_surname = None
        pred_name = None
    phase = "check if correct"
    if row["persona"] == person:
        correct = True
    else:
        correct = False
    phase = "append"
    
    res.append({"True persona": row["persona"], "True name": row["name"], "True surname": row["surname"],
                "Text":row["text"],  "Correct":correct, "Predicted persona": person,
                "Predicted surname": pred_surname, 
                "Predicted name": pred_name})
            
clear_output(wait=True)
print('done '+str(index)+'/'+str(l)+'    '+str(((index)/l)*100)+'%')
result = pd.DataFrame.from_dict(res)
result['Correct'].describe()

done 999/1000    99.9%
CPU times: total: 5min 34s
Wall time: 5min 34s


count     1000
unique       2
top       True
freq       618
Name: Correct, dtype: object

In [22]:
def evaluation(row):
    import pandas as pd
    import numpy as np
    import re
    import os
    import sys
    sys.path.insert(0, 'Desktop/Unimi/information retriveal/project/corpus (Alfio)/inforet-project/')
    #os.chdir('Desktop/Unimi/information retriveal/project/corpus (Alfio)/inforet-project/')
    import segment
    import segment_sur
    import segment2_sur
    from collections import Counter
    
    
    #l = len(text.index)
    res = []
    errors = []
    
    global regex
    regex = re.compile("[^a-zA-Z -']")
    

    #for index, row in text.iterrows():
    #DEBUG
    #if index%100==0:
    #clear_output(wait=True)
    #print('done '+str(index)+'/'+str(l)+'    '+str((index/l)*100)+'%')
    #DEBUG
    global phase
    try:
        person = find_person(row['text'], row['date'], filtered_personnel)
    except Exception as ex:
        #errors.append({'index':index, 'text':row['text'], 'phase':phase, 'ex':ex})
        errors.append({'text':row['text'], 'phase':phase, 'ex':ex})
        return pd.DataFrame.from_dict(res), pd.DataFrame.from_dict(errors)
        #continue
    predicted = filtered_personnel.loc[filtered_personnel['persona'] == person]
    if person is not None:
        pred_surname = predicted.iloc[0]['surname']
        pred_name = predicted.iloc[0]['name']
    else:
        pred_surname = None
        pred_name = None
    if row["persona"] == person:
        correct = True
    else:
        correct = False

    res.append({"True persona": row["persona"], "True name": row["name"], "True surname": row["surname"],
                "Text":row["text"],  "Correct":correct, "Predicted persona": person,
                "Predicted surname": pred_surname, 
                "Predicted name": pred_name})

    #clear_output(wait=True)
    #print('Done')
    return pd.DataFrame.from_dict(res), pd.DataFrame.from_dict(errors)

In [23]:
import ipyparallel as ipp
rc = ipp.Client()
#dv = rc[:]
#dv.push({'filtered_personnel':filtered_personnel})
rc.ids

[]

In [27]:
rc.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [29]:
dview = rc[:]

In [ ]:
def ini():
    import os
    import sys
    sys.path.insert(0, 'Desktop/Unimi/information retriveal/project/corpus (Alfio)/inforet-project/')
    os.chdir('Desktop/Unimi/information retriveal/project/corpus (Alfio)/inforet-project/')
    import stanza
    it_nlp = stanza.Pipeline('it', processors='tokenize,mwt,ner')
    
dview.apply(ini)
dview.wait()

In [28]:
rc[:].push(dict(
    words = words,
    P = P,
    correction = correction,
    candidates = candidates,
    known = known,
    edits1 = edits1,
    edits2 = edits2,
    filtered_personnel=filtered_personnel,
    find_person = find_person,
    find_by_surname = find_by_surname,
    find_by_surname_2 = find_by_surname_2,
    find_by_name = find_by_name,
    SURNAME = SURNAME,
    NAME = NAME,
    TEXT = TEXT,
    WORDS = WORDS
))
view = rc.load_balanced_view()

TypeError: cannot pickle '_thread.lock' object

In [33]:
#ar = view.map_async(evaluation, [row for index, row in split_df[0].iterrows()])

Errors with:

split_df[470]

835

In [ ]:
%%time

for i, df in enumerate(split_df[0:2]):
    ar = view.map_async(evaluation, [row for index, row in df.iterrows()])
    ar.wait() # Wait until all tasks are done.
    #async_results.append(ar)
    results = pd.DataFrame({'A' : []})
    for res in ar:
        if len(res[1])>0:
            print(res[1])
        results = pd.concat([results, res[0]], ignore_index=True)
    results = results.drop(columns=['A'])
    file_name = './resultStanza/res'+str(i)+'.csv'
    results.to_csv(file_name,index=False)
    clear_output(wait=True)
    print('Done: ' + str(i) + '/' + str(len(split_df)))


Done: 0/1000


602

In [26]:
text2

,text,persona,date,surname,name
0,"HELFER, Sottosegretario di Stato per i l turis...",http://dati.camera.it/ocd/persona.rdf/p8740,1960-12-07 00:00:00,HELFER,RENZO
1,BERNIERI. Non si t ra t ta di un emendamento ...,http://dati.camera.it/ocd/persona.rdf/p6820,1953-01-10 00:00:00,BERNIERI,ANTONIO
2,"ROSA JERVOLINO RUSSO, Presidente della I Commi...",http://dati.camera.it/ocd/persona.rdf/p37000,2000-07-25 00:00:00,JERVOLINO RUSSO,ROSA
3,MARCO BOATO. Si t r a t t a di u n a mate r i ...,http://dati.camera.it/ocd/persona.rdf/p20770,1993-11-11 00:00:00,BOATO,MARCO
4,BUONTEMPO TEODORO,http://dati.camera.it/ocd/persona.rdf/p32490,1994-05-05 00:00:00,BUONTEMPO,TEODORO
...,...,...,...,...,...
995,SINESIO ed altri (2355) ; l'Istituto nazionale...,http://dati.camera.it/ocd/persona.rdf/p5520,1974-07-23 00:00:00,SINESIO,GIUSEPPE
996,"ROBERTO GIACHETTI, ricordato che l’articolo 5-...",http://dati.camera.it/ocd/persona.rdf/p38550,2003-07-30 00:00:00,ROSSO,ROBERTO
997,Molinari Giuseppe (MARGH-U) ................ 4...,http://dati.camera.it/ocd/persona.rdf/p50069,2004-07-29 00:00:00,MOLINARI,GIUSEPPE MARIO
998,"2723, 2728, 2730, 2734, 2736, 2737, 2740, RUBI...",http://dati.camera.it/ocd/persona.rdf/p21920,1987-10-06 00:00:00,RUBINACCI,GIUSEPPE


In [181]:
pd.set_option("display.max_rows", None)
result[result['Correct'] == False]

,True persona,True name,True surname,Text,Correct,Predicted persona,Predicted surname,Predicted name
0,http://dati.camera.it/ocd/persona.rdf/p300247,GIUSEPPE,GIANNI,GIUSEPPE FANFANI dichiara il voto favorevole d...,False,http://dati.camera.it/ocd/persona.rdf/p300284,FANFANI,GIUSEPPE
1,http://dati.camera.it/ocd/persona.rdf/p50389,GUIDO,POSSA,"GUIDO POSSA, parlando sull’ordine dei lavori, ...",False,http://dati.camera.it/ocd/persona.rdf/p36980,GUIDI,ANTONIO
7,http://dati.camera.it/ocd/persona.rdf/p8420,RICCARDO,GEFTER WONDRICH,"(21606) « GEFTER WONDRICII, DE MICHIELI",False,None,None,None
8,http://dati.camera.it/ocd/persona.rdf/p301576,TERESA MARIA,DI SALVO,DI SALVO C,False,None,None,None
9,http://dati.camera.it/ocd/persona.rdf/p37100,LELIO,LANTELLA,LANTELLA LELIO V V,False,None,None,None
11,http://dati.camera.it/ocd/persona.rdf/p50373,STEFANO,STEFANI,"STEFANO ALLASIA. Signor Presidente, onorevoli ...",False,http://dati.camera.it/ocd/persona.rdf/p302145,ALLASIA,STEFANO
13,http://dati.camera.it/ocd/persona.rdf/p930,GIORGIO,BOGI,"GIORGIO BENVENUTO. Signor Presidente, interven...",False,http://dati.camera.it/ocd/persona.rdf/p50102,BENVENUTO,ANTONIOGIORGIO
16,http://dati.camera.it/ocd/persona.rdf/p8990,CLEMENTE,MAGLIETTA,« MAGLIETTA » . « I sottoscritti chiedono di i...,False,None,None,None
17,http://dati.camera.it/ocd/persona.rdf/p20440,ROBERTO,CICCIOMESSERE,"CICCIOMESSERE, MELLINI, RIPPA,",False,None,None,None
20,http://dati.camera.it/ocd/persona.rdf/p17970,FRANCANTONIO,BIAGGI,« BIAGGI FRANCANTONIO » . « Il sottoscritto ch...,False,None,None,None


In [182]:
doc = it_nlp(text2.iloc[33]['text'])
doc.sentences[0].text
print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')

In [172]:
text2.iloc[i]['text']

'« MAGLIETTA » . « I sottoscritti chiedono di interrogare i l ministro di grazia e giustizia, per sapere se sia fondata la notizia secondo la quale fra le sedi di pretura destinate ad essere soppresse sarebbero comprese Dorgali (Nuoro), Pozzo maggiore, Ploaghe e Ittiri (Sassari), centr i di notevole importanza economico-sociale e per i quali il solito criterio aritmetico del lavoro giudiziario svoltovi non sarebbe davver o sufficiente a giustificare un provvedimento d i soppressione delle preture stesse . (10407) '

In [185]:
i = 36
find_person(text2.iloc[i]['text'], text2.iloc[i]['date'], filtered_personnel)

Text:
PELLEGRINO . quanto meno ingenuo , onorevole Bassi .
PELLEGRINO BASSI
                                             persona     surname       name  \
2131    http://dati.camera.it/ocd/persona.rdf/p19550  PELLEGRINO   GIUSEPPE   
4713      http://dati.camera.it/ocd/persona.rdf/p580       BASSI       ALDO   
17713  http://dati.camera.it/ocd/persona.rdf/p303971  PELLEGRINO  SALVATORE   

      dataInizio   dataFine  carica  nomeOrganoGoverno  
2131  1963-05-09 1968-06-04     NaN                NaN  
4713  1963-05-09 1968-06-04     NaN                NaN  
17713 1963-05-16 1968-06-04     NaN                NaN  


In [186]:
chunk_list = []
doc = it_nlp(text2.iloc[i]['text'])
for ent in doc.sentences[0].ents:
    chunk_list.append(ent.text)
    print(ent.text)
chunked_name = ' '.join(chunk_list)
date = filtered_personnel[(filtered_personnel["dataInizio"] <= text2.iloc[i]['date']) & (filtered_personnel["dataFine"] >= text2.iloc[i]['date'])]
find_by_surname(chunked_name.upper(), date, False)

PELLEGRINO
Bassi


,persona,surname,name,dataInizio,dataFine,carica,nomeOrganoGoverno
2131,http://dati.camera.it/ocd/persona.rdf/p19550,PELLEGRINO,GIUSEPPE,1963-05-09,1968-06-04,NaN,NaN
4713,http://dati.camera.it/ocd/persona.rdf/p580,BASSI,ALDO,1963-05-09,1968-06-04,NaN,NaN
17713,http://dati.camera.it/ocd/persona.rdf/p303971,PELLEGRINO,SALVATORE,1963-05-16,1968-06-04,NaN,NaN


--------------------------

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [11]:
import sys
sys.path.insert(0, 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.7')

In [2]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [10]:
lent = [0,0,0,0,0,0,0,0]
for sur in filtered_personnel['surname'].unique():
    word_list = sur.split()
    l = len(word_list)
    if l < 8:
        lent[l-1] += 1
    else:
        lent[7] += 1
    if l == 6:
        print(sur)
print(lent)

DI SAN MARTINO LORENZATO DI IVREA
[5539, 461, 35, 2, 2, 1, 0, 0]


In [4]:
filtered_personnel['surname'].unique()

array(['RESCIGNO', 'BONOMI', 'RICCI', ..., 'AMIDEI', 'AURICCHIO',
       'FASIOLO'], dtype=object)

In [7]:
for sur in filtered_personnel['surname'].unique():

    print(sur + ' ' + str(len(word_list)))

RESCIGNO 1
BONOMI 1
RICCI 1
RIGAMONTI 1
RIVA 1
ROCCHETTI 1
ROMANATO 1
ROMANO 1
ROSSI 1
RUBINO 1
RUSSO 1
SABATINI 1
SACCENTI 1
SAGGIN 1
SALERNO 1
SALUTARI 1
SAMPIETRO 1
SANZO 1
SAVIO 1
SCARASCIA MUGNOZZA 2
SCARONGELLA 1
SCHIANO 1
SCHIRO' 1
SCIAUDONE 1
SCIORILLI BORRELLI 2
SECRETO 1
SENSI 1
SERBANDINI 1
SERONI 1
SFORZA 1
SILIPO 1
SILVESTRI 1
SODANO 1
SORGI 1
SPADAZZI 1
SPECIALE 1
STAGNO D'ALCONTRES 2
STORTI 1
STUANI 1
STUCCHI 1
SULOTTO 1
SURACI 1
TAROZZI 1
TITOMANLIO 1
TOGNONI 1
TOMMASI 1
TOSATO 1
TRABUCCHI 1
TREMELLONI 1
TROMBETTA 1
TUDISCO 1
TURCO 1
VALANDRO 1
VECCHIETTI 1
VECCHIO 1
VENEGONI 1
VERONESI 1
VILLANI 1
VIVIANI 1
WALTER 1
ZAMPONI 1
ZANFAGNINI 1
ZAPPA 1
ZERBI 1
MAROTTA 1
TRIMARCHI 1
AVOLIO 1
PASINI 1
ALBA 1
GOZZI 1
CECATI 1
FERRETTI 1
LECCISI 1
SCALIA 1
BALDELLI 1
BOZZI 1
DE PASCALIS 2
VIZZINI 1
DEL VESCOVO 2
MARCHESI 1
MARILLI 1
BELLONI 1
PAGLIUCA 1
GULLO 1
LATORRE 1
AGOSTA 1
CERRETI 1
ANGELUCCI 1
DE VITA 2
DE BIAGI 2
CARPANO MAGLIOLI 2
BENVENUTI 1
CORTESE 1
CAPPA 1
NITTI 1
